## Fetch raw data aggregations of daily stock data for the ticker list from polygon.io.

In [15]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os

In [ ]:
# Define the API details
api_key = 'xxx' # add your api key but preferred to save it into environment

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

# Define the path where the files will be saved
save_path = "C:\\Users\\user\\Data\\rawAggs" # example saving path on device.

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Signal handler for graceful shutdown
def signal_handler(sig, frame):
    print("You pressed Ctrl+C!")
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

# Function to retrieve and save aggregate data for a given ticker over a date range
def get_aggs_for_ticker(ticker, start_date, end_date):
    """Retrieve aggregates for a given ticker and date range"""
    client = RESTClient(api_key=api_key)  # Initialize client

    aggs = []
    for day in weekdays_between(start_date, end_date):
        for a in client.list_aggs(ticker, 1, "day", day, day, limit=50000):
            aggs.append(a)

    # Save the data to a compressed .pickle.lz4 file. We wan it compressed as there might be a lot of data.
    filename = os.path.join(save_path, f"{ticker}-aggs-{start_date}_to_{end_date}.pickle.lz4")
    with open(filename, "wb") as file:
        try:
            compressed_data = lz4.frame.compress(pickle.dumps(aggs))
            file.write(compressed_data)
        except TypeError as e:
            logging.error(f"Serialization Error for {ticker}: {e}")

    logging.info(f"Downloaded aggs for {ticker} from {start_date} to {end_date} and saved to {filename}")

# Function to generate all weekdays between two dates
def weekdays_between(start_date, end_date):
    """Generate all weekdays between start_date and end_date"""
    day = start_date
    while day <= end_date:
        if day.weekday() < 5:  # Only Monday to Friday
            yield day
        day += timedelta(days=1)

def main():
    start_date = datetime.date(2022, 1, 1)  # Start date: Beginning of 2022
    end_date = datetime.date(2024, 9, 30)   # End date: 2024-09-30

    # Load tickers from CSV file (assumes the CSV file has columns 'Ticker' and 'Market-Cap')
    csv_file_path = "C:\\Users\\user\\filename.csv"  # Update this path with your actual file location. Same location as dataTickers save path.
    df_tickers = pd.read_csv(csv_file_path, delimiter=',', header=0)

    # Extract tickers from the 'Ticker' column
    symbols = df_tickers['Ticker'].tolist()

    # Use ThreadPoolExecutor to download data for each ticker in parallel. No idea what this is but seems important ;P.
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_aggs_for_ticker, symbol, start_date, end_date) for symbol in symbols]
        
        # Optional: to make sure each task completes
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # This will raise any exceptions that occurred during execution
            except Exception as e:
                logging.error(f"Error fetching data: {e}")

if __name__ == "__main__":
    main()